In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

In [2]:
spark = SparkContext(master="local", appName="Dataframes")
sqlContext = SQLContext(spark)

In [3]:
def removeHeader(index, iterator):
    return iter(list(iterator)[1:])

In [4]:
!ls ./data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [5]:
path = "./data/"

###### Dataframe juegos.csv

In [6]:
game_schema = StructType([
    StructField("game_id", IntegerType(), False),
    StructField("year", StringType(), False),
    StructField("season", StringType(), False),
    StructField("city", StringType(), False)
])

game_df = sqlContext.read.schema(game_schema) \
    .option("header", "true").csv(path+"juegos.csv")

In [7]:
game_df.show(5)

+-------+-----------+------+------+
|game_id|       year|season|  city|
+-------+-----------+------+------+
|      1|1896 Verano|  1896|Verano|
|      2|1900 Verano|  1900|Verano|
|      3|1904 Verano|  1904|Verano|
|      4|1906 Verano|  1906|Verano|
|      5|1908 Verano|  1908|Verano|
+-------+-----------+------+------+
only showing top 5 rows



In [8]:
deportistaOlimpicoRDD = spark.textFile(path+"deportista.csv") \
    .map(lambda l: l.split(","))
deportistaOlimpicoRDD2 = spark.textFile(path+"deportista2.csv") \
    .map(lambda l: l.split(","))

In [9]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [10]:
deportistaOlimpicoRDD2.count()

67785

In [11]:
# Union de datos
deportistaOlimpicoRDD = deportistaOlimpicoRDD \
    .union(deportistaOlimpicoRDD2)

In [12]:
#Removemos header primero y luego si procesamos
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(removeHeader)

In [13]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l : (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
))

In [14]:
schema = StructType([
    StructField("sport_player", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("gender", IntegerType(), False),
    StructField("age", IntegerType(), False),
    StructField("height", IntegerType(), False),
    StructField("weight", FloatType(), False),
    StructField("team_id", IntegerType(), False)
])

In [15]:
sports_players_df = sqlContext.createDataFrame(deportistaOlimpicoRDD,schema)

In [16]:
sports_players_df.show(5)

+------------+--------------------+------+---+------+------+-------+
|sport_player|                name|gender|age|height|weight|team_id|
+------------+--------------------+------+---+------+------+-------+
|           1|           A Dijiang|     1| 24|   180|  80.0|    199|
|           2|            A Lamusi|     1| 23|   170|  60.0|    199|
|           3| Gunnar Nielsen Aaby|     1| 24|     0|   0.0|    273|
|           4|Edgar Lindenau Aabye|     1| 34|     0|   0.0|    278|
|           5|Christine Jacoba ...|     2| 21|   185|  82.0|    705|
+------------+--------------------+------+---+------+------+-------+
only showing top 5 rows



###### Dataframe paises.csv

In [17]:
paises_RDD = spark.textFile(path + "paises.csv") \
    .map(lambda l: l.split(","))

In [18]:
#Removemos header
paises_RDD = paises_RDD.mapPartitionsWithIndex(removeHeader)

In [19]:
#Mapeo:
paises_RDD = paises_RDD.map(lambda l : (
    int(l[0]),
    l[1],
    l[2]
))

In [20]:
schema = StructType([
    StructField("team_id",IntegerType(),False),
    StructField("team_name",StringType(),False),
    StructField("initials",StringType(),False)
])

In [21]:
teams_df = sqlContext.createDataFrame(paises_RDD,schema)

In [22]:
teams_df.show(5)

+-------+--------------------+--------+
|team_id|           team_name|initials|
+-------+--------------------+--------+
|      1|         30. Februar|     AUT|
|      2|A North American ...|     MEX|
|      3|           Acipactli|     MEX|
|      4|             Acturus|     ARG|
|      5|         Afghanistan|     AFG|
+-------+--------------------+--------+
only showing top 5 rows



###### Dataframe deportes.csv

In [23]:
!ls ./data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [24]:
!head -n 5 ./data/deporte.csv

deporte_id,deporte
1,Basketball
2,Judo
3,Football
4,Tug-Of-War


In [25]:
sport_schema = StructType([
    StructField("sport_id", IntegerType(), False),
    StructField("sport_name", StringType(), False)
])

sport_df = sqlContext.read.schema(sport_schema) \
    .option("header", "true").csv(path+"deporte.csv")

In [26]:
sport_df.show(5)

+--------+-------------+
|sport_id|   sport_name|
+--------+-------------+
|       1|   Basketball|
|       2|         Judo|
|       3|     Football|
|       4|   Tug-Of-War|
|       5|Speed Skating|
+--------+-------------+
only showing top 5 rows



###### Dataframe evento.csv

In [27]:
!head -n 5 ./data/evento.csv

evento_id,evento,deporte_id
1,Basketball Men's Basketball,1
2,Judo Men's Extra-Lightweight,2
3,Football Men's Football,3
4,Tug-Of-War Men's Tug-Of-War,4


In [28]:
event_schema = StructType([
    StructField("event_id", IntegerType(), False),
    StructField("event", StringType(), False),
    StructField("sport_id", StringType(), False)
])

event_df = sqlContext.read.schema(event_schema) \
    .option("header","true").csv(path+"evento.csv")

In [29]:
event_df.show(5)

+--------+--------------------+--------+
|event_id|               event|sport_id|
+--------+--------------------+--------+
|       1|Basketball Men's ...|       1|
|       2|Judo Men's Extra-...|       2|
|       3|Football Men's Fo...|       3|
|       4|Tug-Of-War Men's ...|       4|
|       5|Speed Skating Wom...|       5|
+--------+--------------------+--------+
only showing top 5 rows



###### Dataframe resultados.csv

In [30]:
!head -n 5 ./data/resultados.csv

resultado_id,medalla,deportista_id,juego_id,evento_id
1,NA,1,39,1
2,NA,2,49,2
3,NA,3,7,3
4,Gold,4,2,4


In [31]:
result_schema = StructType([
    StructField("result_id ",IntegerType(),False),
    StructField("medal",StringType(),False),
    StructField("sport_player_id",IntegerType(),False),
    StructField("game_id",IntegerType(),False),
    StructField("event_id", IntegerType(), False)
])
result_df = sqlContext.read.schema(result_schema) \
    .option("header","true").csv(path+"resultados.csv")

In [32]:
result_df.show(5)

+----------+-----+---------------+-------+--------+
|result_id |medal|sport_player_id|game_id|event_id|
+----------+-----+---------------+-------+--------+
|         1|   NA|              1|     39|       1|
|         2|   NA|              2|     49|       2|
|         3|   NA|              3|      7|       3|
|         4| Gold|              4|      2|       4|
|         5|   NA|              5|     36|       5|
+----------+-----+---------------+-------+--------+
only showing top 5 rows



### Impresion de Schemas
Visualizar comportamiento del schema

In [33]:
sport_df.printSchema()

root
 |-- sport_id: integer (nullable = true)
 |-- sport_name: string (nullable = true)



In [34]:
sports_players_df.printSchema()

root
 |-- sport_player: integer (nullable = false)
 |-- name: string (nullable = false)
 |-- gender: integer (nullable = false)
 |-- age: integer (nullable = false)
 |-- height: integer (nullable = false)
 |-- weight: float (nullable = false)
 |-- team_id: integer (nullable = false)



### Renombrado y borrado de columnas

In [35]:
from pyspark.sql.functions import *

In [36]:
sports_players_df = sports_players_df.withColumnRenamed("gender", "sex").drop("height")

In [37]:
sports_players_df.show(5)

+------------+--------------------+---+---+------+-------+
|sport_player|                name|sex|age|weight|team_id|
+------------+--------------------+---+---+------+-------+
|           1|           A Dijiang|  1| 24|  80.0|    199|
|           2|            A Lamusi|  1| 23|  60.0|    199|
|           3| Gunnar Nielsen Aaby|  1| 24|   0.0|    273|
|           4|Edgar Lindenau Aabye|  1| 34|   0.0|    278|
|           5|Christine Jacoba ...|  2| 21|  82.0|    705|
+------------+--------------------+---+---+------+-------+
only showing top 5 rows



In [38]:
sports_players_df.printSchema()

root
 |-- sport_player: integer (nullable = false)
 |-- name: string (nullable = false)
 |-- sex: integer (nullable = false)
 |-- age: integer (nullable = false)
 |-- weight: float (nullable = false)
 |-- team_id: integer (nullable = false)



##### SQL Functions
Existen algunas funciones similares a SQL que podemos utilizar
Si usamos col, habilita operaciones como alias

In [39]:
sports_players_df = sports_players_df.select("sport_player","name", col("age")\
                         .alias("age_played"), "team_id")

In [40]:
sports_players_df.show(5)

+------------+--------------------+----------+-------+
|sport_player|                name|age_played|team_id|
+------------+--------------------+----------+-------+
|           1|           A Dijiang|        24|    199|
|           2|            A Lamusi|        23|    199|
|           3| Gunnar Nielsen Aaby|        24|    273|
|           4|Edgar Lindenau Aabye|        34|    278|
|           5|Christine Jacoba ...|        21|    705|
+------------+--------------------+----------+-------+
only showing top 5 rows



In [41]:
sports_players_df.sort("age_played").show()

+------------+--------------------+----------+-------+
|sport_player|                name|age_played|team_id|
+------------+--------------------+----------+-------+
|         224|     Mohamed AbdelEl|         0|    308|
|         487|      Inni Aboubacar|         0|    721|
|         226|Sanad Bushara Abd...|         0|   1003|
|          58|    Georgi Abadzhiev|         0|    154|
|         230|    Moustafa Abdelal|         0|    308|
|         102|   Sayed Fahmy Abaza|         0|    308|
|         260|  Ahmed Abdo Mustafa|         0|   1003|
|         139|George Ioannis Abbot|         0|   1043|
|         281|      S. Abdul Hamid|         0|    487|
|         163|     Ismail Abdallah|         0|   1095|
|         285|Talal Hassoun Abd...|         0|    497|
|         173| Mohamed Abdel Fatah|         0|   1003|
|         179|Ibrahim Saad Abde...|         0|   1003|
|         378|     Angelik Abebame|         0|      0|
|         294|Mohamed Ghulom Ab...|         0|     81|
|         

### Filtrado con Dataframes

In [43]:
sports_players_df = sports_players_df \
    .filter(sports_players_df.age_played != 0)

In [44]:
sports_players_df.sort("age_played").show(5)

+------------+--------------------+----------+-------+
|sport_player|                name|age_played|team_id|
+------------+--------------------+----------+-------+
|       71691|  Dimitrios Loundras|        10|    333|
|       52070|        Etsuko Inada|        11|    514|
|       40129|    Luigina Giavotti|        11|    507|
|       37333|Carlos Bienvenido...|        11|    982|
|       47618|Sonja Henie Toppi...|        11|    742|
+------------+--------------------+----------+-------+
only showing top 5 rows



#### Agg functions:
Podemos ejecutar aggregated functions como min, max, avg, etc

In [48]:
sports_players_df.agg({'age_played': 'min'}).show()

+---------------+
|min(age_played)|
+---------------+
|             10|
+---------------+



In [49]:
sports_players_df.agg({'age_played': 'max'}).show()

+---------------+
|max(age_played)|
+---------------+
|             97|
+---------------+

